In [1]:
!unzip /content/Images_CNNTask.zip

Archive:  /content/Images_CNNTask.zip
   creating: Images_CNNTask/Class0/
 extracting: Images_CNNTask/Class0/CG001_M_L.png  
 extracting: Images_CNNTask/Class0/CG001_M_R.png  
 extracting: Images_CNNTask/Class0/CG002_M_L.png  
 extracting: Images_CNNTask/Class0/CG002_M_R.png  
  inflating: Images_CNNTask/Class0/CG003_M_L.png  
  inflating: Images_CNNTask/Class0/CG003_M_R.png  
  inflating: Images_CNNTask/Class0/CG004_F_L.png  
 extracting: Images_CNNTask/Class0/CG004_F_R.png  
 extracting: Images_CNNTask/Class0/CG005_F_L.png  
 extracting: Images_CNNTask/Class0/CG005_F_R.png  
 extracting: Images_CNNTask/Class0/CG006_F_L.png  
 extracting: Images_CNNTask/Class0/CG006_F_R.png  
 extracting: Images_CNNTask/Class0/CG007_F_L.png  
 extracting: Images_CNNTask/Class0/CG007_F_R.png  
 extracting: Images_CNNTask/Class0/CG008_F_L.png  
 extracting: Images_CNNTask/Class0/CG008_F_R.png  
 extracting: Images_CNNTask/Class0/CG009_M_L.png  
 extracting: Images_CNNTask/Class0/CG009_M_R.png  
 extract

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
batch_size = 32
num_epochs = 10
num_classes = 2

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Splitting data into training and validation sets

train_generator = train_datagen.flow_from_directory(
    'Images_CNNTask',
    target_size=(224, 224),  # Assuming VGG16 input size
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Training data

validation_generator = train_datagen.flow_from_directory(
    'Images_CNNTask',
    target_size=(224, 224),  # Assuming VGG16 input size
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Validation data

Found 266 images belonging to 2 classes.
Found 65 images belonging to 2 classes.


In [5]:
# Step 2: Choice of Pre-trained CNN Model (VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [6]:
# Step 3: Initializing and Modifying the Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [7]:
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Step 4: Fine-tuning with Regularization Techniques
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)


Epoch 1/10
8/8 [==============================] - 184s 23s/step - loss: 0.6003 - accuracy: 0.7436 - val_loss: 0.7349 - val_accuracy: 0.5156
Epoch 2/10
8/8 [==============================] - 181s 23s/step - loss: 0.5741 - accuracy: 0.6667 - val_loss: 0.4422 - val_accuracy: 0.7500
Epoch 3/10
8/8 [==============================] - 181s 23s/step - loss: 0.4817 - accuracy: 0.7564 - val_loss: 0.4283 - val_accuracy: 0.7500
Epoch 4/10
8/8 [==============================] - 182s 23s/step - loss: 0.4278 - accuracy: 0.8248 - val_loss: 0.4129 - val_accuracy: 0.7188
Epoch 5/10
8/8 [==============================] - 181s 23s/step - loss: 0.4463 - accuracy: 0.7778 - val_loss: 0.4174 - val_accuracy: 0.8594
Epoch 6/10
8/8 [==============================] - 178s 23s/step - loss: 0.4024 - accuracy: 0.7778 - val_loss: 0.3978 - val_accuracy: 0.6875
Epoch 7/10
8/8 [==============================] - 179s 23s/step - loss: 0.3684 - accuracy: 0.8248 - val_loss: 0.3487 - val_accuracy: 0.7969
Epoch 8/10
8/8 [====

In [11]:
# Step 6: Evaluation and Comparison
# Evaluate the model on test data
test_generator = train_datagen.flow_from_directory(
    'Images_CNNTask',
    target_size=(224, 224),  # Assuming VGG16 input size
    batch_size=batch_size,
    class_mode='categorical')

Found 331 images belonging to 2 classes.


In [12]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

11/11 [==============================] - 199s 18s/step - loss: 0.3152 - accuracy: 0.8610
Test Accuracy: 0.861027181148529


In [13]:
# Predict classes for test data
y_pred = model.predict(test_generator)
y_true = test_generator.classes

11/11 [==============================] - 199s 18s/step


In [14]:
precision = precision_score(y_true, np.argmax(y_pred, axis=1))
recall = recall_score(y_true, np.argmax(y_pred, axis=1))
f1 = f1_score(y_true, np.argmax(y_pred, axis=1))

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.7237354085603113
Recall: 0.7654320987654321
F1 Score: 0.744
